In [1]:
import tensorflow as tf
import os
import pickle
import numpy as np

CIFAR_DIR = "./data/cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [ ]:
def load_data(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding='latin1')
        return data['data'], data['labels']

class CifarData:
    # 对于训练集需要每次选取时对数据进行shuffle，因为在训练时要进行w参数的更新，shuffle可以减少数据之间的相互依赖关系
    # 对于测试数据集只计算loss和accuracy，无需shuffle
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            for item, label in zip(data, labels):
                if label in [0, 1]: # 这里是二分类，所以只将第一类和第二类的数据和标签保存下来
                    all_data.append(item)
                    all_labels.append(label)
        self._data = np.vstack(all_data) # 这里将保存好的数据合成一个矩阵
        self._labels = np.hstack(all_labels)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0 # 数据集遍历位置指示器
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        # 生成一个制定范围的shuffle整数序列
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch size examples as a batch"""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data() # 如果end已经大于data size的话说明数据已经遍历完，所以将指示器归零
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception('have no more examples')
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator:end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]
train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

batch_data, batch_labels = train_data.next_batch(10)


In [3]:
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int32, [None])

# 这里是单个神经元，所以是二分类，这里w的维度为[3072,1]
# 如果是多个神经元，这里就是w的维度就是[3072,x],输出向量size为x，对应着每一个类的预测概率
# （3072，1)
w = tf.get_variable('w', [x.get_shape()[-1], 1],
                   initializer=tf.random_normal_initializer(0,1))
# (1, )
b = tf.get_variable('b', [1],
                   initializer=tf.constant_initializer(0.0))
# [None,3072] * [3072, 1] = [None, 1]
y_ = tf.matmul(x, w) + b
# 计算出y等于1的概率值，使用sigmoid函数
#[None, 1]
p_y_1 = tf.sigmoid(y_)

# y.shape = [None]
y_reshaped = tf.reshape(y, (-1,1))
y_reshaped_float = tf.cast(y_reshaped, tf.float32)
loss = tf.reduce_mean(tf.square(y_reshaped_float - p_y_1))

# bool
predict = p_y_1 > 0.5
# [1,0,1,0...]
correct_prediction = tf.equal(tf.cast(predict, tf.int32), y_reshaped)
# 为了计算均值，这里将预测的int值转为float类型
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)


In [ ]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 1000

with tf.Session() as sess:
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val,  _ = sess.run(
            [loss, accuracy, train_op], 
            feed_dict={x: batch_data, y: batch_labels})
        if i % 500 == 0:
            print('[Train] step: %d, loss: %4.5f, acc: %4.5f'\
                  %(i, loss_val, acc_val))